# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier as T
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import MultinomialNB as bayes
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.datasets import make_classification

from razdel import tokenize as rt

In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
def razdel_tokenizer(text):
    return [token.text for token in list(rt(text))]

In [4]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [5]:
vectorizer1 = TfidfVectorizer (tokenizer = razdel_tokenizer)
X1 = vectorizer1.fit_transform(train.comment)
X1_test = vectorizer1.transform(test.comment)
vectorizer1.get_feature_names_out()

c:\Users\Alex\scoop\apps\python\3.12.6\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array(['!', '!!', '!!!', ..., 'ёмкостью', 'ёта', 'ётавских'], dtype=object)

In [6]:
vectorizer2 = TfidfVectorizer () #tokenizer = default
X2 = vectorizer2.fit_transform(train.comment)
X2_test = vectorizer2.transform(test.comment)
vectorizer2.get_feature_names_out()

array(['00', '000', '0015', ..., 'ёмкостью', 'ёта', 'ётавских'],
      dtype=object)

In [7]:
y1 = train.toxic.values
y1_test = test.toxic.values
y2 = train.toxic.values
y2_test = test.toxic.values

In [8]:
clf = RFC(min_samples_leaf = 2, class_weight = 'balanced')
clf.fit(X1, y1)
preds1 = clf.predict(X1_test)
print(classification_report(y1_test, preds1, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.84      0.88      0.86       962
         1.0       0.74      0.65      0.69       480

    accuracy                           0.81      1442
   macro avg       0.79      0.77      0.78      1442
weighted avg       0.80      0.81      0.80      1442



In [9]:
clf = RFC(min_samples_leaf = 2, class_weight = 'balanced')
clf.fit(X2, y2)
preds2 = clf.predict(X2_test)
print(classification_report(y2_test, preds2, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.83      0.88      0.85       962
         1.0       0.73      0.63      0.67       480

    accuracy                           0.80      1442
   macro avg       0.78      0.76      0.76      1442
weighted avg       0.79      0.80      0.79      1442



Встроенный токенизатор побеждает

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [10]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [11]:
vectorizerL = TfidfVectorizer (min_df=4, max_df=0.1, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))
X_for_L = vectorizerL.fit_transform(train.comment)
X_for_L_test = vectorizerL.transform(test.comment)
#vectorizerTF.get_feature_names_out()

In [12]:
vectorizerB = CountVectorizer (min_df=4, max_df=0.3, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))
X_for_B = vectorizerB.fit_transform(train.comment)
X_for_B_test = vectorizerB.transform(test.comment)
#vectorizerC.get_feature_names_out()

In [13]:
yL = train.toxic.values
yL_test = test.toxic.values
yB = train.toxic.values
yB_test = test.toxic.values

In [14]:
clfL = LogisticRegression(penalty='l2', class_weight= 'balanced')
clfL.fit(X_for_L, yL)
predsL = clfL.predict(X_for_L_test)
print(classification_report(yL_test, predsL, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.89      0.89      0.89       962
         1.0       0.78      0.79      0.78       480

    accuracy                           0.85      1442
   macro avg       0.83      0.84      0.84      1442
weighted avg       0.85      0.85      0.85      1442



In [15]:
clfB = bayes(alpha=1, force_alpha=True)
clfB.fit(X_for_B, yB)
predsB = clfB.predict(X_for_B_test)
print(classification_report(yB_test, predsB, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.85      0.93      0.89       962
         1.0       0.82      0.68      0.74       480

    accuracy                           0.84      1442
   macro avg       0.84      0.80      0.81      1442
weighted avg       0.84      0.84      0.84      1442



In [16]:
predsB_proba = clfB.predict_proba(X_for_B_test)
predsL_proba = clfL.predict_proba(X_for_L_test)

In [17]:
from collections import Counter
from pprint import pprint

In [18]:
counterB = Counter({comm: prob[1] for comm, prob in zip(data.comment, predsB_proba)})
B_fin = counterB.most_common(10)
pprint(B_fin)

[('Ок. Раз уж крахоборничать то пусть будет максимум не - 1 Бар, а - 1,013 '
  'Бар. При полной откачке, скажем, воздуха из емкости, на емкость будет '
  'воздействовать атмосферное давление, и большее разрежение, чем оно давит, '
  'ваккумметр не покажет. То есть максимум - 1 атмосфера, или 1,013 Бар.\n',
  np.float64(1.0)),
 ('Тиньков! СУКА! Ты по что кредита мне не даешь?! Впрочем...не больно то и '
  'хотелось! ....))))',
  np.float64(1.0)),
 ('во-во. побочки в одиссее явно подтянули. намного интереснее проходить и так '
  'остро уже этот гринд не ощущается.',
  np.float64(1.0)),
 ('Барином хорошо было быть, в 17-ом веке. Учи себе французкий. Читай книжки. '
  'Не жизнь - малина.)',
  np.float64(1.0)),
 ('Бате недавно ставили, многим уже в 40 нужно. А если не повезло с зубами, то '
  'и того раньше. Надеюсь, вам повезло :р',
  np.float64(0.99999999999892)),
 ('Как по мне, салденс какую-то хуйню(разумеется, все это 10из10 по сравнению '
  'с массмаркетом) начал варить. Десятки сорто

In [19]:
counterL = Counter({comm: prob[1] for comm, prob in zip(data.comment, predsL_proba)})
L_fin = counterL.most_common(10)
pprint(L_fin)

[('Переведите этот поток сознания хотя бы хохляцкий, плиз.\n',
  np.float64(0.9885916374155893)),
 ('Я бы на месте цыганки выбил бы телефон из рук и лицо бы поцарапал,за это '
  'ничего не будет,просто административный штраф,а пикабушник запомнит,что '
  'дерьмом по жизни быть нельзя и судить других никто не имеет права.\n',
  np.float64(0.9840106318453373)),
 ('я вот не пойму, я у тебя совета что-ли из раза в раз прошу?\n',
  np.float64(0.9777982005413092)),
 ('Так там по сути никто и не работает дольше 3-х лет для стажа в типа сириус '
  'конторе. У нас в филиале роскосого так: после универа за 15-20к работают во '
  'всяких лабораториях, потом уебывают на вольные хлеба. Постоянно там только '
  'старичье совкового пошива и сотни начальников, 90 которых даже на работу не '
  'ходят, чисто зп получают.\n',
  np.float64(0.9700229400504651)),
 ('Ты из какого века блядь?', np.float64(0.9689419517586311)),
 ('да не, там вполне современная, относительно, техника. в квартирах разбитых '
  '

In [20]:
same = list()
for i in L_fin:
    for j in B_fin:
        if i==j and j not in same:
            same.append(j)
same

[]

одинаковых текстов нет, тексты действительно токсичные

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [23]:
import numpy 

In [21]:
B_importance = clfB.feature_log_prob_[1]
featuresB = vectorizerB.get_feature_names_out()
toxicB = B_importance.argsort()[-5:][::-1]
[featuresB[i] for i in toxicB]

['это', 'просто', 'тебе', 'всё', 'вообще']

In [24]:
L_importance = numpy.abs(clfL.coef_[0])
featuresL = vectorizerL.get_feature_names_out()
toxicL = L_importance.argsort()[-5:][::-1]
[featuresL[i] for i in toxicL]

['хохлы', 'хохлов', 'тебе', 'дебил', 'нахуй']

In [26]:
vectorizerRFC = TfidfVectorizer (min_df=4, max_df=0.1, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))
X_for_RFC = vectorizerRFC.fit_transform(train.comment)
X_for_RFC_test = vectorizerRFC.transform(test.comment)

vectorizerT = TfidfVectorizer (min_df=4, max_df=0.1, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))
X_for_T = vectorizerT.fit_transform(train.comment)
X_for_T_test = vectorizerT.transform(test.comment)

In [27]:
yRFC = train.toxic.values
yRFC_test = test.toxic.values
yT = train.toxic.values
yT_test = test.toxic.values

In [28]:
clfRFC = RFC(min_samples_leaf = 2, class_weight = 'balanced')
clfRFC.fit(X_for_RFC, yRFC)
predsRFC = clfRFC.predict(X_for_RFC_test)

clfT = T(min_samples_leaf = 2, class_weight = 'balanced')
clfT.fit(X_for_T, yT)
predsT = clfT.predict(X_for_T_test)

In [29]:
RFC_importance = clfRFC.feature_importances_
featuresRFC = vectorizerRFC.get_feature_names_out()
toxicRFC = RFC_importance.argsort()[-5:][::-1]
[featuresRFC[i] for i in toxicRFC]

['очень', 'хохлы', 'тебе', 'хохлов', 'спасибо']

In [30]:
T_importance = clfT.feature_importances_
featuresT = vectorizerT.get_feature_names_out()
toxicT = T_importance.argsort()[-5:][::-1]
[featuresT[i] for i in toxicT]

['тебе', 'очень', 'хохлы', 'лет', 'всё']